In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
import random
import argparse
from PIL import Image
from skimage.transform import rotate
from sklearn.model_selection import train_test_split
from scipy.linalg import sqrtm
from os.path import isfile, join
import csv
import datetime
import sunrise
import pytz
from tzwhere import tzwhere
tzwhere = tzwhere.tzwhere()

Using TensorFlow backend.
/home/trifunov/anaconda3/envs/dmm/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
path = "yourdatapath"

In [3]:
def normalize(variable):
        normalized_var = (variable - np.mean(variable)) / np.std(variable)
        return normalized_var

In [4]:
def get_sunrise_sunset(lat, lon, start_datetime):
        """
        A function that determines the sunrise and the sunset times based on the geographical coordinates of the FLUXNET site
        
        Input:
        latitude - integer value of the latitude of the given FLUXNET site 
        longitude - integer value of the longitude of the given FLUXNET site
        start_datetime - timestamp variable of the FLUXNET site containing integers of the format YYYYMMDDHHMinMin
        (e.g. 202101291857)

        Output:
        sunrise_time - integer value of the time of the day when the sun rises on a given FLUXNET site of the form HMinMin (e.g. 531)
        sunset_time - integer value of the time of the day when the sun sets on a given FLUXNET site of the form HHMinMin (e.g. 2104)
        """
        timezone_str = tzwhere.tzNameAt(lat, lon)
        timezone = pytz.timezone(timezone_str)
        M = len(start_datetime)
        sunrise_time = np.zeros(M)
        sunset_time = np.zeros(M)
        days = [None] * M

        for i in range(M):
            s = sunrise.sun(lat=lat,long=lon)
            year = int(start_datetime[i]) // 100000000
            month = (int(start_datetime[i]) % 100000000) // 1000000
            day = (int(start_datetime[i]) % 1000000) // 10000
            day_input = datetime.datetime(year, month, day)
            srise = s.sunrise(when=day_input)
            sset = s.sunset(when=day_input)
            srise = pytz.utc.localize(day_input.replace(hour=srise.hour, minute=srise.minute))
            sset = pytz.utc.localize(day_input.replace(hour=sset.hour, minute=sset.minute))
            sunrise_time[i] = int(srise.astimezone(timezone).strftime('%H%M'))
            sunset_time[i] = int(sset.astimezone(timezone).strftime('%H%M'))
            days[i] = day_input
        return sunrise_time, sunset_time

In [5]:
def day_night(variable, timestamp, sunrise_time, sunset_time):
    """
    A function that takes a time series variable as input and outputs its daytime and nighttime values as separate variables.

    Input:
    variable - time series variable starting at midnight
    timestamp - array of integers denoting the date and time in the format YYYYMMDDHHMinMin
    sunrise_time - integer hour of day when the daylight starts (6:30 --> 630)
    sunset_time - integer hour of day when the daylight ends (21:00 --> 2130)

    Output:
    variable_day - daytime values of the input variable
    variable_night - nighttime values of the input variable
    """

    n = len(variable)
    # m = n // 24
    variable_day = np.empty(n)
    variable_day[:] = np.NaN
    variable_night = np.empty(n)
    variable_night[:] = np.NaN
    time_of_day = np.array(timestamp % 10000)
    for i in range(n):
        if (time_of_day[i] >= sunset_time[i] + 100) or (time_of_day[i] < sunrise_time[i] - 100): #100 denotes 1 hour, making sure there is not sunlight for photosynthesis
            variable_night[i] = variable[i]
        else:
            variable_day[i] = variable[i]

    return variable_night, variable_day

In [6]:
def load_eco_vars_csv(site_name):
    """
    A function for loading data from a FLUXNET site in csv format.

    Input:
    site_name - string, name of the data file
    Output:
    Ecological variables of interest from that site
    """
    
    real_eco_data_csv = pd.read_csv(path+site_name, skip_blank_lines=False)
    
    #load desired FLUXNET variables
    radiation = real_eco_data_csv['SW_IN_F_MDS']
    temperature = real_eco_data_csv['TA_F_MDS']
    respiration = real_eco_data_csv['RECO_NT_VUT_USTAR50']
    GPP = real_eco_data_csv['GPP_NT_VUT_USTAR50']
    NEE = real_eco_data_csv['NEE_VUT_USTAR50']
    precipitation = real_eco_data_csv['P_F']
    VPD = real_eco_data_csv['VPD_F_MDS']
    start_datetime = real_eco_data_csv['TIMESTAMP_START']
    SWC = real_eco_data_csv['SWC_F_MDS_1']
    SWC_T = real_eco_data_csv['TS_F_MDS_1']
    WS = real_eco_data_csv['WS_F']
    WD = real_eco_data_csv['WD']

    # load site coordinates from .csv file
    coordinates = pd.read_csv("fluxnet-site-coordinates.csv", sep='\t', header = None)
    len_sites = coordinates.iloc[:,0].shape[0]
    site_name_short = site_name.split('/')[-1]
    for i in range(len_sites):
        if coordinates.iloc[i,0] == site_name_short[4:10]:
            lat = coordinates.iloc[i,2]
            lon = coordinates.iloc[i,3]
    return radiation, temperature, respiration, GPP, NEE, precipitation, VPD, float(lat), float(lon), start_datetime, SWC, SWC_T, WS, WD

In [7]:
fluxnet_sites_HH = [f for f in os.listdir("path_of_your_fluxnet_sites")]
for site in fluxnet_sites_HH:
    radiation, temperature, respiration, GPP, NEE, precipitation, VPD, latitude, longitude, start_datetime, SWC, T_soil, WS, WD = load_eco_vars_csv("FLUXNETsites-from-ANN-partitioning-paper/" + site)

    radiation = radiation.to_numpy()
    temperature = temperature.to_numpy()
    respiration = respiration.to_numpy()
    GPP = GPP.to_numpy()
    NEE = NEE.to_numpy()
    precipitation = precipitation.to_numpy()
    VPD = VPD.to_numpy()
    SWC = SWC.to_numpy()
    T_soil = T_soil.to_numpy()
    WS = WS.to_numpy()
    WD = WD.to_numpy()
    start_datetime = start_datetime.astype(float)

    ts_length = len(radiation)

    radiation_normalized = normalize(radiation)
    temperature_normalized = normalize(temperature)
    #GPP_normalized = normalize(GPP_noisy) #for synthetic data
    GPP_normalized = normalize(GPP)
    respiration_normalized = normalize(respiration)
    precipitation_normalized = normalize(precipitation)
    NEE_normalized = normalize(NEE)
    VPD_normalized = normalize(VPD)
    SWC_normalized = normalize(SWC)
    T_soil_normalized = normalize(T_soil)
    WS_normalized = normalize(WS)
    WD_normalized = normalize(WD)

    years = int(ts_length / (365 * 48)) #half-hourly data
    T = 30*48
    num_eco_vars = 14

    def create_sliding_window_eco_dataset(list_months, measure, ts_length, R_g, T_air, GPP, R_eco, NEE, Pr, VPD, latitude, longitude, start_datetime, SWC, T_soil, WS, WD, num_vars, sample_length, shift, reps):
        """
        Function creating a train, validation, test split of the input ecological variables using sliding windows to augment the data
        ***************
        list_months - list of integers from 1 to 12 in an increasing order denoting months of the year to be included in the dataset
        measure - integer denoting if one year of data is measured daily (1), hourly (24) or half-hourly (48)
        ts_length - integer denoting the length of the FLUXNET site used
        R_g - time series of global radiation
        T_air - time series of air temperature
        GPP - time series of global primary production
        R_eco - time series of ecosystem respiration
        NEE - time series of net ecosystem exchange
        Pr - time series of precipitation
        VPD - time series of vapor pressure deficit
        latitude - float of latitude of the fluxnet site the data is coming from
        longitude - float longitude of the fluxnet site the data is coming from
        start_datetime - timestamp_start variable of the fluxnet site noting integer of the format YYYYMMDDHHMinMin (year, month, day, hour, minute)
        SWC - time series of soil water content
        T_soil - time series of soil temperature
        WS - time series of wind speed
        WD - time series of wind direction
        ANN_Reco - time series of Reco estimated by the ANN partitioning method (NN_part in the paper)
        num_vars - integer denoting the number of ecological variables and their components (such as seasonal cycle) in the new dataset
        sample_length - integer denoting the length of the sample in days
        shift - integer denoting the number of days two sliding windows are apart from each other
        reps - integer denoting the number of replications of the data

        """
        M = len(list_months)
        J = 30 // shift
        T = sample_length * measure
        years = int(ts_length / (365 * measure))
        data_train = np.zeros([M*(years-2)*J*reps, T, num_vars])
        data_valid = np.zeros([M * J * reps, T, num_vars])
        sunrise_time, sunset_time = get_sunrise_sunset(latitude, longitude, start_datetime)

        for rep in range(reps):
            noise_1 = np.random.normal(0, 0.5, ts_length)
            np.random.seed(1 + rep * 100)
            noise_2 = np.random.normal(0, 0.5, ts_length)
            np.random.seed(10 + rep *200)
            noise_3 = np.random.normal(0, 0.5, ts_length)
            np.random.seed(100 + rep *300)
            noise_4 = np.random.normal(0, 0.5, ts_length)
            for i in range(M):
                for year in range(years):
                    for j in range(J):
                        if year < (years - 2):
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 0] = R_g[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 1] = T_air[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 2], _ = day_night(NEE[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], start_datetime[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], sunrise_time[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], sunset_time[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure])
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 3], _ = day_night(GPP[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], start_datetime[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], sunrise_time[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], sunset_time[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure])
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 4] = GPP[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 5] = NEE[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 6] = Pr[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 7] = VPD[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 8] = R_eco[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]

                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 9] = start_datetime[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]

                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 10] = SWC[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 11] = T_soil[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 12] = WS[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_train[(((years-2) * i + year) * J + j) * reps + rep, :, 13] = WD[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                        # when using an unseen year for the test    
                        elif year == years-2:
                            data_valid[(J * i + j) * reps + rep, :, 0]= R_g[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 1] = T_air[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 2], _ = day_night(NEE[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], start_datetime, sunrise_time, sunset_time)
                            data_valid[(J * i + j) * reps + rep, :, 3], _ = day_night(GPP[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure], start_datetime, sunrise_time, sunset_time)
                            data_valid[(J * i + j) * reps + rep, :, 4] = GPP[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 5] = NEE[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 6] = Pr[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 7] = VPD[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 8] = R_eco[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]

                            data_valid[(J * i + j) * reps + rep, :, 9] = start_datetime[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]

                            data_valid[(J * i + j) * reps + rep, :, 10] = SWC[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 11] = T_soil[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 12] = WS[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]
                            data_valid[(J * i + j) * reps + rep, :, 13] = WD[(year * 365 + 30 * list_months[i] + shift * j) * measure:(year * 365 + 30 * list_months[i] + sample_length + shift * j) * measure]


    return data_train, data_valid

    #picking the summer months June, July, August, September
    data_train, data_valid = create_sliding_window_eco_dataset([6, 7, 8, 9], 48, ts_length, radiation, temperature, GPP, respiration, NEE, precipitation, VPD, latitude, longitude, start_datetime, SWC, T_soil, WS, WD, num_eco_vars, 7, 2, 1)

    site_name = site.split('.')[0]
    np.savez(path + "prepared_FLUXNET_sites-numpy/" + site_name + "_June-to-September", data_train, data_valid, data_test)